# Path

In [47]:
import sys
import os

project_path = r'/home/craxiss/Documents/projects/kapsul_etkinlik_v2'

os.chdir(project_path)
sys.path.append(project_path)

# Imports

In [48]:
from helpers import column_formatter, utf8_convert
from charset_normalizer import detect
from config import data_path as path
from datetime import datetime
from json import dump, load
import pandas as pd
import openpyxl

# Read

In [49]:
sheetnames = openpyxl.load_workbook(path+'1-raw/yoklama.xlsx').sheetnames

appeals = pd.read_excel(path+'/2-cleaned/basvuru.xlsx')

# Sync Sheet Names

In [50]:
lesson101 = dict()
for i in range(appeals.shape[0]):
    row = appeals.iloc[i:i+1]
    for col in appeals.columns[4:]:
        lessons = str(row[col].iloc[0]).split(',')
        for lesson in lessons:
            lesson101[lesson.strip(' ')] = lesson.strip()


del lesson101['nan'] # delete unnamed
for k,v in {'IOT':'IOT 101', 'Finans':'Finans 101'}.items(): # set unmatched lessons
    lesson101[k] = v

synced_sheet_names = dict()
for sheetname in sheetnames:
    sheetname_stripped = sheetname.strip().lower()
    for lesson in lesson101:
        if sheetname_stripped in lesson.lower():
            synced_sheet_names[sheetname] = lesson101[lesson]

In [51]:
print('Non-Match Sheets', set(sheetnames) - set(synced_sheet_names))

synced_sheet_names['Ardunio '] = 'Arduino 101'
synced_sheet_names['Proje Yönetimi'] = 'Proje Döngüsü Yönetimi 101'

# Metaverse is empty
del synced_sheet_names['Metaverse ']

Non-Match Sheets {'Proje Yönetimi', 'Ardunio '}


# Read Sheets

In [52]:
sheets = dict()
for k, v in synced_sheet_names.items():
    sheets[v] = pd.read_excel(path+'1-raw/yoklama.xlsx', sheet_name=k)

# Sync Column Names

In [53]:
all_columns = set()

for data in sheets:
    data = sheets[data]
    all_columns = set.union(all_columns, set(data.columns))


for data_col in sheets:
    data = sheets[data_col]

    sheet_cols = list()
    for col in data.columns:
        if col == "EĞİTİMCİSİ":
            sheet_cols.append("EĞİTİMCİ")

        else:
            sheet_cols.append(col.strip())

    data.columns = sheet_cols

all_columns = set()

for data in sheets:
    data = sheets[data]
    all_columns = set.union(all_columns, set(data.columns))


# Get Roll Called Person per Lesson

In [54]:
roll_call_user_dict = dict()

for sheet in sheets:
    df = sheets[sheet]
    df['İSİM SOYİSİM'] = column_formatter(df['İSİM'] + df['SOYİSİM'])

    for name in df['İSİM SOYİSİM']:
        name = utf8_convert(name)
        if name not in roll_call_user_dict:
            roll_call_user_dict[name] = list()
        roll_call_user_dict[name].append(sheet)


# Format Lessons

In [55]:
lesson_info = dict()
for sheet in sheets:
    df = sheets[sheet]
    df['CİNSİYET'] = df['CİNSİYET'].str.replace('KIZ', 'KADIN')
    row = df.iloc[0:1]

    lesson_info[sheet] = {'TARİH': str(row['TARİH'].iloc[0]),
                          'GÜN': row['GÜN'].iloc[0], 'EĞİTİMCİ': row['EĞİTİMCİ'].iloc[0]}

    lesson_info[sheet]['BAŞLANGIÇ SAATİ'], lesson_info[sheet]['BİTİŞ SAATİ'] = row['SAAT'].iloc[0].split(
        '-')

lesson_info['Finans 101']['TARİH'] = '2022-06-02 00:00:00'

# Format Times

In [56]:
months = ['ocak', 'şubat', 'mart', 'nisan', 'mayıs', 'haziran',
          'temmuz', 'ağustos', 'eylül', 'ekim', 'kasım', 'aralık']

days = ["pazartesi", "salı", "çarşamba",
        "perşembe", "cuma", "cumartesi", "pazar"]

lesson_info_df = pd.DataFrame(lesson_info).T

In [57]:
month, weekday = list(), list()
for date in lesson_info_df['TARİH']:
    date = datetime.strptime(date.replace(' 00:00:00', ''), "%Y-%m-%d")
    month.append(months[date.month-1]), weekday.append(days[date.weekday()])

lesson_info_df['GÜN'] = weekday
lesson_info_df['AY'] = month

lesson_info_df.sort_values(by='TARİH', inplace=True)

# Charset Normalization

In [58]:
for sheet in sheets:
    df = sheets[sheet][['İSİM SOYİSİM', 'CİNSİYET']]
    for col in range(df.shape[1]):
        for i in range(df.shape[0]):
            df.iat[i, col] = utf8_convert(df.iat[i, col])
    
    sheets[sheet] = df

# Write

## XLSX

In [59]:
sheets['Hikaye Anlatıcılığı 101']

,İSİM SOYİSİM,CİNSİYET
0,everettfrost,ERKEK
1,henryreeves,KADIN
2,christinahirota,ERKEK
3,elizabethboyle,ERKEK
4,keniathornburg,KADIN
5,andreawynne,KADIN
6,justinward,KADIN
7,lukekorth,KADIN
8,kimfoley,KADIN
9,raymonddixon,ERKEK


In [60]:
with pd.ExcelWriter(path+'2-cleaned/yoklama.xlsx', mode='w') as writer:
    for sheet in sheets:
        df = sheets[sheet]
        df.columns = ['name', 'gender']
        df.to_excel(writer, sheet_name=sheet, index=False)

    
    lesson_info_df.to_excel(writer, sheet_name='General Info')